<a href="https://colab.research.google.com/github/ronald-hk-chung/ML/blob/main/1_Training%20Simple%20Machine%20Learning%20Alogoritms%20for%20Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Training Simple Machine Learning Algorithms for Classification

## Note book Summary
We will make use of two algorithmically described machine learning algorithms for classification: the perceptron and adaptive linear neurons to classify different flower specifies in teh Iris dataset.

*   Building an understanding of machine learning algorithms
*   Using Pandas, NumPy and Matplotlib to read in, process and visualize data
*   Implementing linear classifiers for 2-class problems in Python

